In [ ]:
import gc
gc.collect()

import sys
sys.path.insert(0, '../')
import logging
logging.basicConfig(level=logging.ERROR)

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from cryptotrader.exchange_api.poloniex import Poloniex
from cryptotrader.envs.trading import BacktestDataFeed, BacktestEnvironment
from cryptotrader.envs.utils import make_balance, convert_to
from cryptotrader.agents import apriori
from cryptotrader.utils import array_normalize, simplex_proj

from time import time, sleep

from bokeh.io import output_notebook
output_notebook()
%matplotlib inline
%load_ext line_profiler

In [ ]:
# Environment setup
obs_steps = 300
period = 30
pairs = ["USDT_BTC", "USDT_ETH", "USDT_LTC", "USDT_XRP", "USDT_XMR", "USDT_ETC", "USDT_DASH"]
init_funds = make_balance(crypto=0.0, fiat=100.0, pairs=pairs)

tapi = Poloniex()
tapi = BacktestDataFeed(tapi, period, load_dir='./data', pairs=pairs, portfolio=init_funds)
# tapi.download_data(end=datetime.timestamp(datetime.utcnow() - timedelta(days=100)),
#                        start=datetime.timestamp(datetime.utcnow() - timedelta(days=200)))
tapi.load_data('/train')

env = BacktestEnvironment(period, obs_steps, tapi, 'backtest_env')
env.add_pairs(pairs)
env.fiat = 'USDT'
obs = env.reset()

In [ ]:
# Training run
# Training params
nb_steps = 50
batch_size = 2
nb_max_episode_steps = 7

agent = apriori.MomentumTrader(ma_span=[3,50], std_span=50, activation=simplex_proj)

hp = {
    'ma1': [2, env.obs_steps],
    'ma2': [2, env.obs_steps],
    'std_span': [2, env.obs_steps],
    'alpha_up': [1e-4, 5e-1],
    'alpha_down': [1e-4, 5e-1]
    }

search_space = {'mean_type':{'simple': hp,
                             'exp': hp,
                             'kama': hp
                             }
                }
constrains = [lambda mean_type, ma1, ma2, std_span, alpha_up, alpha_down: ma1 < ma2]

opt_params, info = agent.fit(env, nb_steps, batch_size, search_space, constrains, nb_max_episode_steps=nb_max_episode_steps)
print("\n", opt_params,"\n", env.status)

# Previous found params:
# {'mean_type': 'kama', 'alpha_down': 0.4128425807825884, 'alpha_up': 0.37140222586733046, 'ma1': 133, 'ma2': 234, 'std_span': 39}

agent.test(env, verbose=True)
env.plot_results();

In [ ]:
# Validation run
# tapi.download_data(end=datetime.timestamp(datetime.now() - timedelta(days=50)),
#                        start=datetime.timestamp(datetime.now() - timedelta(days=100)))
env.tapi.load_data('/eval')
agent.test(env, verbose=True)
env.plot_results();

In [ ]:
# Test run
# tapi.download_data(end=datetime.timestamp(datetime.now()),
#                        start=datetime.timestamp(datetime.now() - timedelta(days=50)))
env.tapi.load_data('/test')
agent.test(env, verbose=True)
env.plot_results();